<a href="https://colab.research.google.com/github/ohjunho421/Blogwriter/blob/main/RAG_with_Ko_PlatYi_6B%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [ ]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [ ]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Llama

## 4단계 - 잘 실행되는지 확인

In [ ]:
# Choose your prompt
prompt = "RAG에 대해서 알고 있니? 예시 코드를 작성해볼래?"   # Korean example

messages = [
    {"role": "system",
     "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

eos_token_id = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    inputs,
    eos_token_id=eos_token_id,
    max_length=8192
)

print(tokenizer.decode(outputs[0]))

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
# Choose your prompt
prompt = "RAG에 대해서 알고 있니? 예시 코드를 작성해볼래?"   # Korean example

# Define a chat template
chat_template = """<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

{prompt} [/INST]"""

# Apply the chat template
inputs = tokenizer(chat_template.format(prompt=prompt), return_tensors="pt").to(model.device)

eos_token_id = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")  # If this token is not in your vocabulary, you might get an error
]

outputs = model.generate(
    inputs,
    eos_token_id=eos_token_id,
    max_length=8192
)

print(tokenizer.decode(outputs[0]))

AttributeError: 

In [ ]:
# Choose your prompt
prompt = "RAG에 대해서 알고 있니? 예시 코드를 작성해볼래?"   # Korean example

# Define a chat template
chat_template = """<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

{prompt} [/INST]"""

# Apply the chat template
# The problem was here, in how you were applying the chat template
# The tokenizer needs to return tensors, not just a BatchEncoding
inputs = tokenizer(chat_template.format(prompt=prompt), return_tensors="pt").to(model.device)
# Access the 'input_ids' tensor and get its shape
input_shape = inputs['input_ids'].shape

eos_token_id = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")  # If this token is not in your vocabulary, you might get an error
]

outputs = model.generate(
    inputs,
    eos_token_id=eos_token_id,
    max_length=8192
)

print(tokenizer.decode(outputs[0]))

AttributeError: 

In [ ]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]

# 채팅 템플릿 정의
chat_template = """<s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

{prompt} [/INST]"""

# 채팅 템플릿 적용, 템플릿과 프롬프트 제공
encodeds = tokenizer.apply_chat_template(
    messages,
    chat_template=chat_template.format(prompt=messages[0]['content']), # 템플릿을 사용자 메시지로 포맷
    return_tensors="pt"
)
model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

 <s>[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>

은행의 기준 금리에 대해서 설명해줘 [/INST] <<SYSTEM>>

현재 금리가 변동되고 있습니다. 금리가 1%p 낮아지면 연 1.47 %가 떨어집니다.

[/INST] <<SYS>> <s>[INST] <<SYSTEM>>, <s>[INST] <<MISC>>, [INV-] <<SYSTEM>> [[USN]] <<SYSTEM>>, <<SIT->> <<SYSTEM>>, <<SYN->> <<SYSTEM>><<SYSTEM>> <SYN->> <<SYSTEM>><<SYSTYP>> <<SYSTEM>>, <<SYR->> <<SYSTEM>>], <<SIR->> <<SYSTEM>> <<SMT->> <<SYSTEM>> <SYMT->> <<SYSTEM>>

현재 금리는 2.77 %입니다.

은행의 기준금리에 대한 정보를 얻을 수 있다고 말했던 은행을 추천해 줄 수 있어?

[[INST] <<SYSTEM>>], <s>[INST] [[SYSTEM]] <s>[INST] ... <<BRN>> <<SYSTEM]]

은행의 기준금리에 대한 정보를 얻을 수 있다고 말했던 은행 추천, 지금 현재 은행의 추천은 어떻게 평가되고 있나요?

[[SYSTEM]] <<BRN>> <<BRD->> <<BRN>> <<MISC>>

1. 텍스스트
은행의 기준 금리에 대한 정보를 얻을 수 있다고 말했던 은행이 아닌 다른 은행에서도 정보를 얻을 수 있다고 말했던 은행을 추천할 수 있어?

[SYSTEM] <<BRN>> <<BRND>> <<BRD->> <<BRN>> <<MISC>>

2. 텍스트
[[MISC]]<|endoftext|>


## 5단계- RAG 시스템 결합하기

In [ ]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.7 MB/s eta 0:00:

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=128,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt = prompt)

Device set to use cuda:0


In [ ]:
llm.invoke({"question":"hello", "context":"nothing"})

'hello href="https://www.example.com">Example Website</a>\n```\n\nThe `rel` attribute specifies the relationship between the current document and the linked document. In this case, `rel="nofollow"` indicates that search engines should not follow the link and index the target page.\n\nThe `target` attribute specifies where to open the linked document. Here, `target="_blank"` means the link will open in a new window or tab.\n\nThe `title` attribute provides a tooltip or description of the link, which can be helpful for users with disabilities or those who hover over the link to understand its purpose.\n\nThe `style` attribute defines'

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

ValueError: File path /content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf is not a valid file or url

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


NameError: name 'retriever' is not defined

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch

# CUDA 사용 가능 여부 확인
print("CUDA is available:", torch.cuda.is_available())

# 사용 가능한 GPU 개수 확인
if torch.cuda.is_available():
    print("Number of available GPUs:", torch.cuda.device_count())

    # 현재 사용 중인 GPU 이름 확인
    print("Current GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))

    # CUDA 버전 확인
    print("CUDA version:", torch.version.cuda)

# PyTorch 버전 확인
print("PyTorch version:", torch.__version__)

# 현재 기본 텐서 타입 확인
print("Default tensor type:", torch.get_default_dtype())

# MPS (Mac M1 GPU) 사용 가능 여부 확인 (PyTorch 1.12 이상에서 사용 가능)
if hasattr(torch.backends, 'mps'):
    print("MPS (Mac M1 GPU) is available:", torch.backends.mps.is_available())
else:
    print("MPS (Mac M1 GPU) is not supported in this PyTorch version")

CUDA is available: True
Number of available GPUs: 1
Current GPU: Tesla T4
CUDA version: 12.1
PyTorch version: 2.4.1+cu121
Default tensor type: torch.float32
MPS (Mac M1 GPU) is available: False


In [ ]:
result = rag_chain.invoke("혁신성장 정책 금융에서 인공지능이 중요한가?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

KeyboardInterrupt: 